## VIDEO TO FRAMES

Esta parte del código requiere la ruta donde se aloja el video, y genera los frames de los videos para el Método 1 y Método 2, no es necesario recortar los videos, o eliminar los frames (el código hace una limpieza  y no procesa los primeros y los últimos frames de cada video, es decir tomar solo los frames entre el 40% y el 80 % del video para evitar movimientos al principio o al final)

Solo cambiar la ruta del video

In [ ]:
import cv2
import os

# Carpeta con los videos a los que se les quiere extraer los frames
folder_path = r"D:\Tecnicas_observacionales\Seeing\Test_videos"
filenames = [file for file in os.listdir(folder_path)]
for filename in filenames:
    filename = filename[:-4]
    print(filename)
    video_path = os.path.join(folder_path, f"{filename}.MOV")

    video_name = os.path.splitext(os.path.basename(video_path))[
        0
    ]  # Extraer el nombre del archivo para usar como nombre de la carpeta

    output_folder = os.path.join(folder_path, "frames", video_name)  # Crear una carpeta con el nombre del video si no existe
    if not os.path.exists(output_folder):
        try:
            os.makedirs(output_folder)
        except OSError:
            print("Error: Creating directory of data")
            exit(1)

    # Abrir el video
    cam = cv2.VideoCapture(video_path)

    # Inicializar el contador de frames
    currentframe = 0

    print(f"\n Inicio de la extracción de frames")

    while True:
        # Leer un frame del video
        ret, frame = cam.read()

        if ret:
            # Si hay frames disponibles, continuar creando imágenes
            name = os.path.join(output_folder, f"frame{currentframe}.jpg")
            # print(f"Creating... {name}") #Mostrar los frames creados

            # Guardar la imagen extraída
            cv2.imwrite(name, frame)

            # Incrementar el contador de frames
            currentframe += 1
        else:
            print("\n Extracción de frames completada")
            print(f"\n {currentframe} frames generados")
            print(f"          ....            ")
            break

    # Liberar todos los recursos y cerrar ventanas una vez terminado
    cam.release()
    cv2.destroyAllWindows()

## Frames Analysis

Aquí se determina la distancia entre los spot de las estrellas del método 1 y el método 2. 

Se requiere insertar la ruta donde se guardaron los frames

Sarazin & Roddier (1990), definen la varianza en términos del movimiento total como:

$$
\sigma^{2} = 2B_{\alpha}(0,0) = 0.358 \left(\frac{\lambda}{r_0}\right)^{5/3} \left(\frac{\lambda}{D}\right)^{1/3}

$$

Donde $D$ es el diámetro de las aperturas 4.2 cm, $r_0$ es el parámetro de Fried y $\lambda$ es la longitud de onda, en este caso el visible , 0.5 $\mu$ m

Seeing :
$$
FWHM = \Delta \theta = 0.98 \lambda / r_0
$$

In [1]:
Dhole = 4.2  # diametro de la apertura  en mm
dsep = 14.4  # diametro de la separacion de las aperturas en mm
lamb = 0.00005  # longitud de onda en micrometros (cm)

In [2]:
"""________________Distance_IMG______________
Programa que construye el histograma de frecuencias de la separación 
entre dos "spots" (objetos) en el DIMM."""

"""_________________Libraries________________________"""
import cv2
import os
import glob
import numpy as np
from scipy.stats import norm
from scipy.spatial import distance as dist
from astropy.stats import mad_std
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import re
from tqdm import tqdm

"""__________________________________________________"""


"""__________________FUNCTIONS_______________________"""
# Definir una función con parámetros de Open CV
def identify_stars_and_distance(image_path, thresh, plot=False):
    # Cargar imagen con filtro en escala de grises
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    imagef = image.flatten()
    # Datos de la imagen sin los pixs menores a 5 (fondo presuntamente)
    #image_nob = imagef[np.where(imagef>5)]

    # Calcular la desviación estándar (ruido de fondo) de la imagen sin el fondo
    std, median = mad_std(image), np.median(image)
    
    # Aplicar un umbral para binarizar la imagen y destacar los objetos (estrellas) Este parámetro se puede variar según la intensidad de la estrella
    _, thresh = cv2.threshold(image, thresh*median , 255, cv2.THRESH_BINARY)

    # Encontrar los contornos de los objetos (estrellas)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar unicamente los dos contornos más grandes detectados (asumiendo que son las estrellas)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:2]

    # Obtener los centros de las dos estrellas
    star_centers = []  #crear un vector para guardar los centros
    for contour in contours:
        M = cv2.moments(contour) #calcula los momentos del contorno, función de Open CV
        if M["m00"] != 0:  # moo es el area del contorno
            cX = int(M["m10"] / M["m00"])  #m10 es el momento espacial para calcular las coordenadas del centroide
            cY = int(M["m01"] / M["m00"])  #m01 es el momento espacial para calcular las coordenadas del centroide
            star_centers.append((cX, cY)) #guardar las coordenadas de los centros encontrados

    # Calcular la distancia entre las dos estrellas
    if len(star_centers) == 2:
        pixel_distance = dist.euclidean(star_centers[0], star_centers[1])
        if plot:
            # Load the image
            img = mpimg.imread(image_path)
            # Display the image
            plt.imshow(img)
            # print(star_centers)
            plt.plot([star_centers[0][0], star_centers[1][0]], [star_centers[0][1], star_centers[1][1]], 'ro', mfc='none')
            plt.tight_layout()
            plt.axis('off')
            # plt.savefig('hm.png', dpi=400)
            plt.show()        

        return(pixel_distance)
    else:
        # print("No se encontraron estrellas.")
        return 0

# Definir una funcion para extraer el numero del frame del nombre del archivo
def extract_frame_number(filename):
    match = re.search(r'frame(\d+)\.jpg', filename)
    return int(match.group(1)) if match else -1

Corrección por video

Factor de recorte, 6000/1920 = 3.125
Por relación de frame del video con tamaño captura


Unidades radianes o unidades de angulo
Longitudinal o transversal




In [ ]:
"""_____________________MAIN_________________________"""
folder = r"D:\Tecnicas_observacionales\Seeing\frames"  # <--------------AQUI SE CAMBIA LA RUTA DONDE SE ENCUENTRAN LOS FRAMES
output_folder = r"D:\Tecnicas_observacionales\Seeing"  # Carpeta de salida para los archivos .dat

files = np.sort([file for file in os.listdir(folder)])

values = np.zeros((len(files),6), dtype=object)
ff = -1

threshold = []
seeing = []
sigma = []
sigma_sec = []
fried = []
fwhm = []

for filename in files:
    ff+=1

    # Abrir carpeta donde se encuentran los frames del video
    carpeta = f"{folder}/{filename}"
    print(filename)

    # Busqueda de los archivos .jpg
    archivos = glob.glob(carpeta + r"/*.jpg")

    # Ordenar los archivos numéricamente por el número de frame en el nombre asi (0, 1, 2, 3, ...)
    archivos.sort(key=extract_frame_number)

    # Definir los frames que se procesaran, se escogieron unicamente los frames entre el 50% y el 80% para eliminar errores al inicio y fin
    start = int(len(archivos)*0.78)
    end = int(len(archivos)*0.8)

    nombres = []
    for j in archivos:
        if carpeta in j:
            nombres.append(j.replace(carpeta, ""))

    if nombres != []:
        l = len(nombres)
        print(f"Se encontraron {l} frames(.jpg) se procesarán del {start} al {end}")
        print("\n :: CONSTRUYENDO EL HISTOGRAMA :: \n")
    else:
        print("Su carpeta no tiene archivos .jpg")

    distances = []
    plotit = False #<-----------------Cambiar a True para visualizar los frames con las estrellas identificadas
    thrhld = 5     #<-----------------Cambiar el valor de i para ajustar el umbral del threshold

    for k in range(start, end):
        # Usar la función en un frame
        dis = identify_stars_and_distance(archivos[k],thrhld, plotit)

        distances.append(dis)  

        if dis!=0:
            plotit=False
    # print(distances)
    distances = np.array(distances)
    no_stars = len(np.where(distances==0)[0])
    # print(f'No se encontraron estrellas en {no_stars} frames, es decir en el {round(no_stars/(end-start+1)*100,2)}% de ellos')

    distances = distances[np.where(distances!=0)]

    # Ajuste de los datos a una distribución normal
    mu, std = norm.fit(distances)
    # Normalizar los valores del histograma
    weights = np.ones_like(distances) / len(distances)
    # Graficar el histograma con las distancias
    count, bins, ignored = plt.hist(distances, weights=weights, bins=15, alpha=0.6, color="g", edgecolor="black")

    # Graficar la función de densidad de probabilidad (PDF) ajustada
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)

    # Normalizar la PDF para que se ajuste al histograma
    bin_width = bins[1] - bins[0]
    p_normalized = p * bin_width  # Ajuste el área de la PDF al histograma
    plt.plot(x, p_normalized, "k", linewidth=2)

    """_____________________SCALE________________________"""

    # scale_size = 0.358  # segundos de arco por px ('' /px)
    scale_size = 1.12 # segundos de arco por px ('' /px)
    """__________________________________________________"""

    # Mostrar los valores de mu y sigma
    print(f"Media (mu): {mu:.3f} pixeles, Desviación estándar (σ): {std:.3f} pixeles")
    sigma_sc = std*scale_size
    print(f"Considerando el factor de escala {scale_size} la desviación estándar sigma (σ): {sigma_sc:.3f} sec")
    print(f"En la distribución normal el FWHM está dado por (2.355 * σ): {2.355*std:.3F}")

    # Parametro de Fried
    """
    Consideraciones para la ecuación del parámetro de Freid:
    206264.8 el factor de conversion de arcsec to rad ( "/rad)
    se asume un r_0 para un solo sentido, la ecuación transversal
    """
    # r_0 = (((0.358 * (lamb / std) ** 2) ** 3) / Dhole) ** (1 / 5)
    r_0 = (((sigma_sc/206264.8)**2)/((2*lamb**2)*(0.179*Dhole**(-1/3)-0.145*dsep**(-1/3))))**(-3/5)
    print(f"\nEl parámetro de Fried es :{r_0:.2f} cm \n")

    # Seeing sin corregir por cenit
    theta = 0.98 * (lamb / r_0) * 206264.8
    fuwhama = 2.355 * std
    print(f"El valor del seeing sin la corrección al cenit es de: {theta:.3f}")

    # Grafico
    plt.title(r"$\mathrm{Histograma\ de\ distancias\ entre\ estrellas:}\ \mu=%.3f,\ \sigma=%.3f,\ \mathrm{FWHM}=%.3f$" % (mu, std, fuwhama))
    plt.xlabel("Distancia entre los centroides (px)")
    plt.ylabel("Frecuencia")
    plt.text(0.15, 0.9, f"Placa: {scale_size} \"/px", transform=plt.gca().transAxes, fontsize=12, color='black', ha='center')
    plt.show()

    threshold.append((filename, thrhld))
    sigma.append((filename, std))
    sigma_sec.append((filename,sigma_sc))
    fried.append((filename, r_0))
    fwhm.append((filename, fuwhama))
    seeing.append((filename, theta))

# Guardar los datos en un archivo .dat dentro de la misma carpeta
output_file = os.path.join(output_folder, f"seeing_sigma.dat")

with open(output_file, "w") as f:
    f.write("# Filename\tThreshold\tSigma(px)\tSigma(sec)\tr_0(cm)\tFWHM\tSeeing_sin_corregir\n")  # Usar \t para tabulaciones
    for (file_threshold, value_threshold),(file_sigma, value_sigma), (file_sigma_sec, value_sigma_sec), (file_fried, value_fried),(file_fwhm, value_fwhm), (file_seeing, value_seeing) in zip(threshold,sigma, sigma_sec, fried, fwhm, seeing):
        f.write(f"{file_seeing}\t{value_threshold:.3f}\t{value_sigma:.3f}\t{value_sigma_sec:.3f}\t{value_fried:.3f}\t{value_fwhm:.3f}\t{value_seeing:.3f}\n")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from datetime import datetime, timedelta

# Cargar datos
df = pd.read_csv('DIMM_data.txt', sep='\t')

# Convertir las horas a formato datetime ajustando rango
df["Hora_datetime"] = pd.to_datetime(df["Hora"], format="%H:%M")
df["Hora_datetime"] = df["Hora_datetime"].apply(lambda x: x + timedelta(days=1) if x.hour < 18 else x)

# Ordenar por hora
df = df.sort_values(by="Hora_datetime")

# Graficar con barras de error y ajustes solicitados
plt.figure(figsize=(6, 4))
plt.errorbar(
    df["Hora_datetime"],
    df["Seeing"],
    yerr=df["sigma (px) software"],
    fmt="o--",                        # Línea discontinua con marcadores
    ecolor="lightgray",               # Barras de error más claras
    capsize=3,                        # Extensión de las barras de error
    markerfacecolor="none",           # Marcadores sin relleno
    markeredgecolor="black",          # Bordes de los marcadores en negro
    color="black",                    # Línea punteada en negro
    linestyle="--"                    # Estilo de línea discontinua
    
)

# Ajustar formato del eje x
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
plt.xlim([datetime.strptime("18:00", "%H:%M"), datetime.strptime("01:00", "%H:%M") + timedelta(days=1)])
plt.ylim(0, 8.5)

# Agregar etiquetas
plt.xlabel("Hora (UTC - 5)")
plt.ylabel("Seeing (segundos de arco)")

# Agregar cuadrícula con subdivisiones
plt.grid(which="both", alpha=0.4)     # Cuadrícula principal
plt.minorticks_on()                  # Activar ticks menores
plt.grid(which="minor", alpha=0.2, linestyle=":")  # Cuadrícula secundaria con estilo punteado

# Mostrar gráfico
plt.tight_layout()
plt.show()
